<a href="https://colab.research.google.com/github/shreya-14-03/social_web_lab_/blob/main/social_web_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

class TwitterClient(object):
	'''
	Generic Twitter Class for sentiment analysis.
	'''
	def __init__(self):
		'''
		Class constructor or initialization method.
		'''
		# keys and tokens from the Twitter Dev Console
		consumer_key = '6nOaJDLHrlUURkrqMNjHETCws'
		consumer_secret = 'Hw3nhw1ZQlcoexdLT9w0ZIzlbGO9knbQ09nydzwwVQEWeI6XoR'
		access_token = '1366400620060499970-gFOVEQEu5tRx1N8rA9rGivXfEtSDKi'
		access_token_secret = '55MnatoE0z1jWqcmQOPT99Uc63sLSSGwl0BVoobOcEdVQ'

		# attempt authentication
		try:
			# create OAuthHandler object
			self.auth = OAuthHandler(consumer_key, consumer_secret)
			# set access token and secret
			self.auth.set_access_token(access_token, access_token_secret)
			# create tweepy API object to fetch tweets
			self.api = tweepy.API(self.auth)
		except:
			print("Error: Authentication Failed")

	def clean_tweet(self, tweet):
		'''
		Utility function to clean tweet text by removing links, special characters
		using simple regex statements.
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

	def get_tweet_sentiment(self, tweet):
		'''
		Utility function to classify sentiment of passed tweet
		using textblob's sentiment method
		'''
		# create TextBlob object of passed tweet text
		analysis = TextBlob(self.clean_tweet(tweet))
		# set sentiment
		if analysis.sentiment.polarity > 0:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

	def get_tweets(self, query, count = 10):
		'''
		Main function to fetch tweets and parse them.
		'''
		# empty list to store parsed tweets
		tweets = []

		try:
			# call twitter api to fetch tweets
			fetched_tweets = self.api.search(q = query, count = count)

			# parsing tweets one by one
			for tweet in fetched_tweets:
				# empty dictionary to store required params of a tweet
				parsed_tweet = {}

				# saving text of tweet
				parsed_tweet['text'] = tweet.text
				# saving sentiment of tweet
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

				# appending parsed tweet to tweets list
				if tweet.retweet_count > 0:
					# if tweet has retweets, ensure that it is appended only once
					if parsed_tweet not in tweets:
						tweets.append(parsed_tweet)
				else:
					tweets.append(parsed_tweet)

			# return parsed tweets
			return tweets

		except tweepy.TweepError as e:
			# print error (if any)
			print("Error : " + str(e))

def main():
	# creating object of TwitterClient Class
	api = TwitterClient()
	# calling function to get tweets
	tweets = api.get_tweets(query = 'Donald Trump', count = 200)

	# picking positive tweets from tweets
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
	# percentage of positive tweets
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
	# picking negative tweets from tweets
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
	# percentage of negative tweets
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
	# percentage of neutral tweets
	print("Neutral tweets percentage: {} % \
		".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))

	# printing first 5 positive tweets
	print("\n\nPositive tweets:")
	for tweet in ptweets[:10]:
		print(tweet['text'])

	# printing first 5 negative tweets
	print("\n\nNegative tweets:")
	for tweet in ntweets[:10]:
		print(tweet['text'])

if __name__ == "__main__":
	# calling main function
	main()


Positive tweets percentage: 34.375 %
Negative tweets percentage: 18.75 %
Neutral tweets percentage: 46.875 % 		


Positive tweets:
@RobbyRoadsteame @elonmusk WOW!

Don Jr. is 🔥🔥🔥🔥 these days!  Must be the tanned testicles.

Thank you… https://t.co/VJPHvsSZTo
RT @jlinett: @GOPLeader Given Trump Jr texts to Mark Meadows on how to overthrow a fair election, why aren’t you demanding an investigation…
RT @OccupyDemocrats: BREAKING: New York AG Letitia James hits back after Donald Trump attacks her as "racist," says she won't be "bullied o…
RT @newsmax: "[Fmr. Pres. Donald Trump] doesn't just want you to repeat the slogans, he actually wants someone with a plan to put America f…
RT @MaddowBlog: "It is past time for Mike Lee to start fessing up to all he knows about the plot to set aside the results of an honest and…
RT @The65Project: Vanity Fair: A new legal group aimed at disbarring the lawyers who pushed Donald Trump’s election fraud lies in the court…
RT @DavidJHarrisJr: So brave.. so bra